In [1]:
import numpy as np
import pandas as pd
import zipcodes
import addfips
af=addfips.AddFIPS()

In [2]:
file = pd.read_csv(r'\\nas01.itap.purdue.edu\puhome\My Documents\Research\Paper1\Preprocessing_files\zip\zipcodes1.csv', header=0)
zsmall = file["GEOID10"].values
zsmall=zsmall[zsmall!=67260]
zsmall=zsmall[zsmall!=64841]
zsmall=zsmall[zsmall!=64858]
location = zsmall

In [3]:
####################   POPULATION RELATED
data1 = np.zeros((len(location)*3,9),dtype=float)
var2 = pd.read_csv(r'\\nas01.itap.purdue.edu\puhome\My Documents\Research\Paper1\Organised Derived CSVs\400_var2.csv', header=0)
years = [1990,2000,2010]
for cnt in range(0,3):
    year=years[cnt]
    if year==1990:
        file1 = pd.read_csv(r'\\nas01.itap.purdue.edu\puhome\My Documents\Research\Paper1\Preprocessing_files\zip\ZIPCNTY.csv', header=0)
        for cnt2 in range(0, len(location)):
            data1[(cnt*len(location))+cnt2,0]=year-1
            zipcode = location[cnt2]
            A = zipcodes.matching(str(zipcode))
            county = af.get_county_fips(A[0]["county"], A[0]["state"])
            county=int(county)
            data1[(cnt*len(location))+cnt2,1]=county
            data1[(cnt*len(location))+cnt2,2]=zipcode
            file_cnty=file1[file1["ZIPCODE"]==zipcode]        
            var22=var2[var2["YEAR"]==year-1]
            var222=var22[var22["COUNTY"]==county]
            if file_cnty.shape[0]==1:
                data1[(cnt*len(location))+cnt2,3]=file_cnty["P0010001"].values[0]
                #for cnt3 in range(1,8):
                    #data1[(cnt*len(location))+cnt2,5]=data1[(cnt*len(location))+cnt2,5]+file_cnty["P060000"+str(cnt3)].values[0]
                black_denom = 0
                for cnt3 in range(1,6):
                    black_denom=black_denom+file_cnty["P012000"+str(cnt3)].values[0]
                if black_denom!=0:
                    data1[(cnt*len(location))+cnt2,7]=file_cnty["P0120002"].values[0]*100/black_denom
                if data1[(cnt*len(location))+cnt2,3]!=0:
                    #data1[(cnt*len(location))+cnt2,5]=data1[(cnt*len(location))+cnt2,5]*100/data1[(cnt*len(location))+cnt2,3]
                    data1[(cnt*len(location))+cnt2,8]=file_cnty["P0100001"].values[0]*100/data1[(cnt*len(location))+cnt2,3]
            elif file_cnty.shape[0]>1:
                pop = file_cnty["P0010001"].values[0]
                data1[(cnt*len(location))+cnt2,3]=file_cnty["P0010001"].values.sum()
                black_denom = 0
                for cnt3 in range(1,6):
                    black_denom=black_denom+file_cnty["P012000"+str(cnt3)].values.sum()
                if black_denom!=0:
                    data1[(cnt*len(location))+cnt2,7]=file_cnty["P0120002"].values.sum()*100/black_denom
                if data1[(cnt*len(location))+cnt2,3]!=0:
                    data1[(cnt*len(location))+cnt2,8]=file_cnty["P0100001"].values.sum()*100/data1[(cnt*len(location))+cnt2,3]
            else:
                data1[(cnt*len(location))+cnt2,3]=-999
                data1[(cnt*len(location))+cnt2,7]=var222["POP_BLACK"].values[0]
                data1[(cnt*len(location))+cnt2,8]=var222["POP_HISP"].values[0]
            data1[(cnt*len(location))+cnt2,4]=var222["POP_INC"].values[0]
            data1[(cnt*len(location))+cnt2,5]=var222["POP_WORK_AGE"].values[0]
            data1[(cnt*len(location))+cnt2,6]=var222["POP_MALE"].values[0]
    if year==2000:
        file1 = pd.read_csv(r'\\nas01.itap.purdue.edu\puhome\My Documents\Research\Paper1\Preprocessing_files\zip\2000_1.csv', header=0)
        fileid = pd.read_csv(r'\\nas01.itap.purdue.edu\puhome\My Documents\Research\Paper1\Preprocessing_files\zip\2000_ZiptoLogrecno.csv', header=0)
        for cnt2 in range(0, len(location)):
            data1[(cnt*len(location))+cnt2,0]=year-1
            zipcode = location[cnt2]
            A = zipcodes.matching(str(zipcode))
            county = af.get_county_fips(A[0]["county"], A[0]["state"])
            county=int(county)
            data1[(cnt*len(location))+cnt2,1]=county
            data1[(cnt*len(location))+cnt2,2]=zipcode
            file_cnty0=fileid[fileid["ZIP"]==str(zipcode)]
            if file_cnty0.shape[0]==1:
                logrecno = file_cnty0["LOGRECNO"].values[0]
                file_cnty=file1[file1["LOGRECNO"]==logrecno]
                pop=file_cnty["POP"].values[0]
                data1[(cnt*len(location))+cnt2,3]=pop
                if data1[((cnt-1)*len(location))+cnt2,3]==-999:
                    data1[((cnt-1)*len(location))+cnt2,3]=pop
                if data1[((cnt-1)*len(location))+cnt2,3]!=0:
                    data1[(cnt*len(location))+cnt2,4]=0.1*(data1[(cnt*len(location))+cnt2,3]-data1[((cnt-1)*len(location))+cnt2,3])*100/data1[((cnt-1)*len(location))+cnt2,3]
                pop_work=0
                for cnt3 in range(23,37):
                    pop_work=pop_work+file_cnty["80"+str(cnt3)].values[0]
                for cnt3 in range(62,76):
                    pop_work=pop_work+file_cnty["80"+str(cnt3)].values[0]
                pop_male=file_cnty["MALE"].values[0]
                pop_black=file_cnty["BLACK"].values[0]
                pop_hisp=file_cnty["HISP"].values[0]
                if pop!=0:
                    data1[(cnt*len(location))+cnt2,5]=pop_work*100/pop
                    data1[(cnt*len(location))+cnt2,6]=pop_male*100/pop
                    data1[(cnt*len(location))+cnt2,7]=pop_black*100/pop
                    data1[(cnt*len(location))+cnt2,8]=pop_hisp*100/pop
            else:
                var22=var2[var2["YEAR"]==year-1]
                var222=var22[var22["COUNTY"]==county]
                data1[(cnt*len(location))+cnt2,3]=data1[((cnt-1)*len(location))+cnt2,3]
                data1[(cnt*len(location))+cnt2,4]=var222["POP_INC"].values[0]
                data1[(cnt*len(location))+cnt2,5]=var222["POP_WORK_AGE"].values[0]
                data1[(cnt*len(location))+cnt2,6]=var222["POP_MALE"].values[0]
                data1[(cnt*len(location))+cnt2,7]=var222["POP_BLACK"].values[0]
                data1[(cnt*len(location))+cnt2,8]=var222["POP_HISP"].values[0]
    if year==2010:
        file1 = pd.read_csv(r'\\nas01.itap.purdue.edu\puhome\My Documents\Research\Paper1\Preprocessing_files\zip\2010_1.csv', header=0)
        file2 = pd.read_csv(r'\\nas01.itap.purdue.edu\puhome\My Documents\Research\Paper1\Preprocessing_files\zip\2010_4.csv', header=0)
        file3 = pd.read_csv(r'\\nas01.itap.purdue.edu\puhome\My Documents\Research\Paper1\Preprocessing_files\zip\2010_3.csv', header=0)
        fileid = pd.read_csv(r'\\nas01.itap.purdue.edu\puhome\My Documents\Research\Paper1\Preprocessing_files\zip\2010_ZiptoLogrecno.csv', header=0)
        for cnt2 in range(0, len(location)):
            data1[(cnt*len(location))+cnt2,0]=year-1
            zipcode = location[cnt2]
            A = zipcodes.matching(str(zipcode))
            county = af.get_county_fips(A[0]["county"], A[0]["state"])
            county=int(county)
            data1[(cnt*len(location))+cnt2,1]=county
            data1[(cnt*len(location))+cnt2,2]=zipcode
            file_cnty0=fileid[fileid["ZIPCODE"]==(zipcode)]
            if file_cnty0.shape[0]==1:
                logrecno = file_cnty0["LOGRECNO"].values[0]
                file_cnty=file1[file1["LOGRECNO"]==logrecno]
                pop=file_cnty["POP"].values[0]
                data1[(cnt*len(location))+cnt2,3]=pop
                if data1[((cnt-1)*len(location))+cnt2,3]==-999:
                    data1[((cnt-1)*len(location))+cnt2,3]=pop
                    data1[((cnt-2)*len(location))+cnt2,3]=pop                
                if data1[((cnt-1)*len(location))+cnt2,3]!=-999:
                    data1[(cnt*len(location))+cnt2,4]=0.1*(data1[(cnt*len(location))+cnt2,3]-data1[((cnt-1)*len(location))+cnt2,3])*100/data1[((cnt-1)*len(location))+cnt2,3]
                if pop==0:
                    continue
                file_cnty=file2[file2["LOGRECNO"]==logrecno]
                pop_work=0
                for cnt3 in range(8,22):
                    n=str(cnt3)
                    if cnt3<10:
                        n="0"+n
                    pop_work=pop_work+file_cnty["1200"+n].values[0]
                for cnt3 in range(32,46):
                    pop_work=pop_work+file_cnty["1200"+str(cnt3)].values[0]
                data1[(cnt*len(location))+cnt2,5]=pop_work*100/pop
                pop_male=file_cnty["120002"].values[0]
                data1[(cnt*len(location))+cnt2,6]=pop_male*100/pop
                file_cnty=file3[file3["LOGRECNO"]==logrecno]
                data1[(cnt*len(location))+cnt2,7]=file_cnty["BLACK"].values[0]*100/pop
                data1[(cnt*len(location))+cnt2,8]=file_cnty["HISP"].values[0]*100/pop            
            else:
                var22=var2[var2["YEAR"]==year-1]
                var222=var22[var22["COUNTY"]==county]
                data1[(cnt*len(location))+cnt2,3]=data1[((cnt-1)*len(location))+cnt2,3]
                data1[(cnt*len(location))+cnt2,4]=var222["POP_INC"].values[0]
                data1[(cnt*len(location))+cnt2,5]=var222["POP_WORK_AGE"].values[0]
                data1[(cnt*len(location))+cnt2,6]=var222["POP_MALE"].values[0]
                data1[(cnt*len(location))+cnt2,7]=var222["POP_BLACK"].values[0]
                data1[(cnt*len(location))+cnt2,8]=var222["POP_HISP"].values[0]
var2 = pd.DataFrame(data=data1,columns=["YEAR","COUNTY","ZIP","POP","POP_INC","POP_WORK_AGE","POP_MALE","POP_BLACK","POP_HISP"])
var2.to_csv(r'\\nas01.itap.purdue.edu\puhome\My Documents\Research\Paper1\Organised Derived CSVs\400_z0var2.csv')

In [4]:
####################        POPULATION DATA CONVERSION TO YEARLY (LIN INTERPOLATION)
file=pd.read_csv(r'\\nas01.itap.purdue.edu\puhome\My Documents\Research\Paper1\Organised Derived CSVs\400_z0var2.csv')
var2 = pd.read_csv(r'\\nas01.itap.purdue.edu\puhome\My Documents\Research\Paper1\Organised Derived CSVs\400_var2.csv', header=0)
############  R
data1=np.zeros((file.shape[0],6))
for cnt in range(0,file.shape[0]):
    year=file["YEAR"].values[cnt]
    var20=var2[var2["YEAR"]==year]
    county=file["COUNTY"].values[cnt]
    var21=var20[var20["COUNTY"]==county]
    if var21["POP"].values[0]!=0:
        data1[cnt,0]=file["POP"].values[cnt]/var21["POP"].values[0]
    if var21["POP_INC"].values[0]!=0:
        data1[cnt,1]=file["POP_INC"].values[cnt]/var21["POP_INC"].values[0]
    if var21["POP_WORK_AGE"].values[0]!=0:
        data1[cnt,2]=file["POP_WORK_AGE"].values[cnt]/var21["POP_WORK_AGE"].values[0]
    if var21["POP_MALE"].values[0]!=0:
        data1[cnt,3]=file["POP_MALE"].values[cnt]/var21["POP_MALE"].values[0]
    if var21["POP_BLACK"].values[0]!=0:
        data1[cnt,4]=file["POP_BLACK"].values[cnt]/var21["POP_BLACK"].values[0]
    if var21["POP_HISP"].values[0]!=0:
        data1[cnt,5]=file["POP_HISP"].values[cnt]/var21["POP_HISP"].values[0]
######   RC
data2=np.zeros((len(location)*3,6))
for cnt in range(0,2):
    for cnt2 in range(0,len(location)):
        data2[(cnt*len(location))+cnt2,0]=(data1[((cnt+1)*len(location))+cnt2,0]-data1[(cnt*len(location))+cnt2,0])*0.1
        data2[(cnt*len(location))+cnt2,1]=(data1[((cnt+1)*len(location))+cnt2,1]-data1[(cnt*len(location))+cnt2,1])*0.1
        data2[(cnt*len(location))+cnt2,2]=(data1[((cnt+1)*len(location))+cnt2,2]-data1[(cnt*len(location))+cnt2,2])*0.1
        data2[(cnt*len(location))+cnt2,3]=(data1[((cnt+1)*len(location))+cnt2,3]-data1[(cnt*len(location))+cnt2,3])*0.1
        data2[(cnt*len(location))+cnt2,4]=(data1[((cnt+1)*len(location))+cnt2,4]-data1[(cnt*len(location))+cnt2,4])*0.1
        data2[(cnt*len(location))+cnt2,5]=(data1[((cnt+1)*len(location))+cnt2,5]-data1[(cnt*len(location))+cnt2,5])*0.1
#######   LINEAR INTERPOLATION
years=[1989,1999,2009]
cnt0=-1
cnt00=-1
initial=1990
data3=np.zeros(((2012-initial+1)*len(location),9))
for cnt in range(1990,2013):
    year=cnt-1
    if year in years:
        cnt0=cnt0+1
        if year!=years[len(years)-1]:
            cnt00=cnt00+1
        file0=file[file["YEAR"]==year]
        for cnt3 in range(0,9):
            data3[((cnt-initial)*len(location)):((cnt+1-initial)*len(location)),8-cnt3]=file0.values[:,file0.shape[1]-1-cnt3]
        continue
    var20=var2[var2["YEAR"]==year]
    data3[((cnt-initial)*len(location)):((cnt+1-initial)*len(location)),0]=year
    for cnt2 in range(0,len(location)):
        zipcode = location[cnt2]
        A = zipcodes.matching(str(zipcode))
        county = af.get_county_fips(A[0]["county"], A[0]["state"])
        county=int(county)
        data3[((cnt-initial)*len(location))+cnt2,1]=county
        data3[((cnt-initial)*len(location))+cnt2,2]=zipcode
        var21=var20[var20["COUNTY"]==county]
        data3[((cnt-initial)*len(location))+cnt2,3]=var21["POP"].values[0]*(((year-years[cnt0])*data2[(cnt00*len(location))+cnt2,0])+data1[(cnt0*len(location))+cnt2,0])
        if data3[((cnt-initial-1)*len(location))+cnt2,3]!=0:
            data3[((cnt-initial)*len(location))+cnt2,4]=(data3[((cnt-initial)*len(location))+cnt2,3]-data3[((cnt-initial-1)*len(location))+cnt2,3])*100/data3[((cnt-initial-1)*len(location))+cnt2,3]
#         data3[((cnt-initial)*len(location))+cnt2,4]=var21["POP_INC"].values[0]*(((year-years[cnt0])*data2[(cnt00*len(location))+cnt2,1])+data1[(cnt0*len(location))+cnt2,1])
        data3[((cnt-initial)*len(location))+cnt2,5]=var21["POP_WORK_AGE"].values[0]*(((year-years[cnt0])*data2[(cnt00*len(location))+cnt2,2])+data1[(cnt0*len(location))+cnt2,2])
        data3[((cnt-initial)*len(location))+cnt2,6]=var21["POP_MALE"].values[0]*(((year-years[cnt0])*data2[(cnt00*len(location))+cnt2,3])+data1[(cnt0*len(location))+cnt2,3])
        data3[((cnt-initial)*len(location))+cnt2,7]=var21["POP_BLACK"].values[0]*(((year-years[cnt0])*data2[(cnt00*len(location))+cnt2,4])+data1[(cnt0*len(location))+cnt2,4])
        data3[((cnt-initial)*len(location))+cnt2,8]=var21["POP_HISP"].values[0]*(((year-years[cnt0])*data2[(cnt00*len(location))+cnt2,5])+data1[(cnt0*len(location))+cnt2,5])
var2 = pd.DataFrame(data=data3,columns=["YEAR","COUNTY","ZIP","POP","POP_INC","POP_WORK_AGE","POP_MALE","POP_BLACK","POP_HISP"])
var2.to_csv(r'\\nas01.itap.purdue.edu\puhome\My Documents\Research\Paper1\Organised Derived CSVs\400_zvar2.csv')

In [4]:
####################    OTHER FEATURES 3 MAIN YEARS
data1 = np.zeros((len(location)*3,11),dtype=float)
var3 = pd.read_csv(r'\\nas01.itap.purdue.edu\puhome\My Documents\Research\Paper1\Organised Derived CSVs\400_var3.csv', header=0)
years = [1990,2000,2011]
file_air=pd.read_csv(r"\\nas01.itap.purdue.edu\puhome\My Documents\Research\Paper1\Preprocessing_files\zip\airport.csv",parse_dates=['ActivationDate'])
file_air["YEAR"]=file_air['ActivationDate'].dt.year
file_area=pd.read_csv(r"\\nas01.itap.purdue.edu\puhome\My Documents\Research\Paper1\Preprocessing_files\zip\area.csv")
inflation=pd.read_csv(r"\\nas01.itap.purdue.edu\puhome\My Documents\Research\Paper1\Preprocessing_files\inflationrate.csv")
for cnt in range(0,3):
    year=years[cnt]
    if year==1990:
        inflation1=inflation[inflation["YEARS"]==year-1]
        inf_rate = inflation1["VALUE"].values[0]
        file1 = pd.read_csv(r'\\nas01.itap.purdue.edu\puhome\My Documents\Research\Paper1\Preprocessing_files\zip\ZIPCNTY.csv', header=0)
        valuetorent=[]
        for cnt2 in range(0, len(location)):
            data1[(cnt*len(location))+cnt2,0]=year-1
            zipcode = location[cnt2]
            A = zipcodes.matching(str(zipcode))
            county = af.get_county_fips(A[0]["county"], A[0]["state"])
            county=int(county)
            data1[(cnt*len(location))+cnt2,1]=county
            data1[(cnt*len(location))+cnt2,2]=zipcode
            file_cnty=file1[file1["ZIPCODE"]==zipcode]        
            var22=var3[var3["YEAR"]==year-1]
            var222=var22[var22["COUNTY"]==county]        
            
            if zipcode==67260:
                continue
            
            if file_cnty.shape[0]==1:
                pop_18=0
                pop_col=0
                for cnt3 in range(1,8):
                    pop_18=pop_18+file_cnty["P060000"+str(cnt3)].values.sum()
                    if cnt3>4:
                        pop_col=pop_col+file_cnty["P060000"+str(cnt3)].values.sum()
                pop_hs=file_cnty["P0600003"].values.sum()
                data1[(cnt*len(location))+cnt2,3]=pop_col*100/pop_18
                data1[(cnt*len(location))+cnt2,4]=pop_hs*100/pop_18
                data1[(cnt*len(location))+cnt2,8]=float(file_cnty["P080A001"].values[0])*inf_rate
                data1[(cnt*len(location))+cnt2,9]=float(file_cnty["H061A001"].values[0])*inf_rate
                valuetorent.append(data1[(cnt*len(location))+cnt2,9]/var222["MED_RENT"].values[0])
                owner_occ=file_cnty["H0080001"].values[0]
                occupied=file_cnty["H0040001"].values[0]
                data1[(cnt*len(location))+cnt2,10]=owner_occ*100/occupied
            elif file_cnty.shape[0]>1:
                pop=file_cnty["P0010001"].values.sum()
                pop_18=0
                pop_col=0
                for cnt3 in range(1,8):
                    pop_18=pop_18+file_cnty["P060000"+str(cnt3)].values.sum()
                    if cnt3>4:
                        pop_col=pop_col+file_cnty["P060000"+str(cnt3)].values.sum()
                pop_hs=file_cnty["P0600003"].values.sum()
                data1[(cnt*len(location))+cnt2,3]=pop_col*100/pop_18
                data1[(cnt*len(location))+cnt2,4]=pop_hs*100/pop_18
                inc=0
                value=0
                for cnt3 in range(0,file_cnty.shape[0]):
                    pop1=file_cnty["P0010001"].values[cnt3]
                    inc=inc+(pop1*file_cnty["P080A001"].values[cnt3])
                    value=value+(pop1*file_cnty["H061A001"].values[0])
                data1[(cnt*len(location))+cnt2,8]=inc*inf_rate/pop
                data1[(cnt*len(location))+cnt2,9]=value*inf_rate/pop
                valuetorent.append(data1[(cnt*len(location))+cnt2,9]/var222["MED_RENT"].values[0])
                owner_occ=file_cnty["H0080001"].values.sum()
                occupied=file_cnty["H0040001"].values.sum()
                data1[(cnt*len(location))+cnt2,10]=owner_occ*100/occupied
            else:
                data1[(cnt*len(location))+cnt2,3]=var222["COLLEGE_COMP"].values[0]
                data1[(cnt*len(location))+cnt2,4]=var222["HIGHSCHOOL_COMP"].values[0]
                data1[(cnt*len(location))+cnt2,8]=var222["MED_INC"].values[0]
                data1[(cnt*len(location))+cnt2,9]=(var222["MED_RENT"].values[0]*np.mean(valuetorent))
                data1[(cnt*len(location))+cnt2,10]=var222["HOME_OWNERSHIP"].values[0]            
            data1[(cnt*len(location))+cnt2,5]=var222["UNEMPLOYMENT_RATE"].values[0]
            file_air_county=file_air[(file_air["ZIPCODE"]==str(location[cnt2])) & (file_air["YEAR"]<year)]
            data1[(cnt*len(location))+cnt2,6]=int(file_air_county.shape[0])
            file_area_zip=file_area[file_area["ZCTA5"]==zipcode]
            data1[(cnt*len(location))+cnt2,7]=file_area_zip["AREALANDPT"].values[0]/1000000

    if year==2000:
        inflation1=inflation[inflation["YEARS"]==year-1]
        inf_rate = inflation1["VALUE"].values[0]
        college=[8,9,10,16,17,18,24,25,26,32,33,34,40,41,42,49,50,51,57,58,59,65,66,67,73,74,75,81,82,83]
        hs=[6,14,22,30,38,47,55,63,71,79]
        fileid = pd.read_csv(r'\\nas01.itap.purdue.edu\puhome\My Documents\Research\Paper1\Preprocessing_files\zip\2000_ZiptoLogrecno.csv', header=0)
        file1 = pd.read_csv(r'\\nas01.itap.purdue.edu\puhome\My Documents\Research\Paper1\Preprocessing_files\zip\2000_24.csv', header=0)
        file2 = pd.read_csv(r'\\nas01.itap.purdue.edu\puhome\My Documents\Research\Paper1\Preprocessing_files\zip\2000_4.csv', header=0)
        file3 = pd.read_csv(r'\\nas01.itap.purdue.edu\puhome\My Documents\Research\Paper1\Preprocessing_files\zip\2000_6.csv', header=0,usecols=["LOGRECNO","INC"])
        file4 = pd.read_csv(r'\\nas01.itap.purdue.edu\puhome\My Documents\Research\Paper1\Preprocessing_files\zip\2000_60.csv', header=0,usecols=["LOGRECNO","VALUE"])
        file5 = pd.read_csv(r'\\nas01.itap.purdue.edu\puhome\My Documents\Research\Paper1\Preprocessing_files\zip\2000_56.csv', header=0,usecols=["LOGRECNO","71","72"])
        var22=var3[var3["YEAR"]==year-1]
        valuetorent=[]
        for cnt2 in range(0, len(location)):
            data1[(cnt*len(location))+cnt2,0]=year-1
            zipcode = location[cnt2]
            A = zipcodes.matching(str(zipcode))
            county = af.get_county_fips(A[0]["county"], A[0]["state"])
            county=int(county)
            data1[(cnt*len(location))+cnt2,1]=county
            data1[(cnt*len(location))+cnt2,2]=zipcode
            file_cnty0=fileid[fileid["ZIP"]==str(zipcode)]
            var222=var22[var22["COUNTY"]==county]
            
            if zipcode==67260:
                continue
            
            if file_cnty0.shape[0]==1:
                logrecno = file_cnty0["LOGRECNO"].values[0]
                file_cnty=file1[file1["LOGRECNO"]==logrecno]
                pop=file_cnty["25001"].values[0]
                pop_col=0
                for cnt3 in college:
                    n=str(cnt3)
                    if cnt3<10:
                        n="0"+str(cnt3)
                    pop_col=pop_col+file_cnty["250"+n].values[0]
                pop_hs=0
                for cnt3 in hs:
                    n=str(cnt3)
                    if cnt3<10:
                        n="0"+str(cnt3)
                    pop_hs=pop_hs+file_cnty["250"+n].values[0]
                data1[(cnt*len(location))+cnt2,3]=pop_col*100/pop
                data1[(cnt*len(location))+cnt2,4]=pop_hs*100/pop
                file_cnty=file2[file2["LOGRECNO"]==logrecno]
                if (file_cnty["43003"].values[0]+file_cnty["43010"].values[0])!=0:
                    data1[(cnt*len(location))+cnt2,5]=(file_cnty["43007"].values[0]+file_cnty["43014"].values[0])*100/(file_cnty["43003"].values[0]+file_cnty["43010"].values[0])
                ########  large difference in county and ZIP code unemployment rate
                if abs(data1[(cnt*len(location))+cnt2,5]-data1[((cnt-1)*len(location))+cnt2,5])>20:
                    data1[((cnt-1)*len(location))+cnt2,5]=data1[(cnt*len(location))+cnt2,5]
                file_cnty=file3[file3["LOGRECNO"]==logrecno]
                data1[(cnt*len(location))+cnt2,8]=float(file_cnty["INC"].values[0])*inf_rate
                file_cnty=file4[file4["LOGRECNO"]==logrecno]
                data1[(cnt*len(location))+cnt2,9]=float(file_cnty["VALUE"].values[0])*inf_rate
                valuetorent.append(data1[(cnt*len(location))+cnt2,9]/var222["MED_RENT"].values[0])
                file_cnty=file5[file5["LOGRECNO"]==logrecno]
                data1[(cnt*len(location))+cnt2,10]=file_cnty["72"].values[0]*100/file_cnty["71"].values[0]
            else:                        
                data1[(cnt*len(location))+cnt2,3]=var222["COLLEGE_COMP"].values[0]
                data1[(cnt*len(location))+cnt2,4]=var222["HIGHSCHOOL_COMP"].values[0]
                data1[(cnt*len(location))+cnt2,5]=var222["UNEMPLOYMENT_RATE"].values[0]
                data1[(cnt*len(location))+cnt2,8]=var222["MED_INC"].values[0]
                data1[(cnt*len(location))+cnt2,9]=(var222["MED_RENT"].values[0]*np.mean(valuetorent))
                data1[(cnt*len(location))+cnt2,10]=var222["HOME_OWNERSHIP"].values[0]
            file_air_county=file_air[(file_air["ZIPCODE"]==str(location[cnt2])) & (file_air["YEAR"]<year)]
            data1[(cnt*len(location))+cnt2,6]=int(file_air_county.shape[0])
            file_area_zip=file_area[file_area["ZCTA5"]==zipcode]
            data1[(cnt*len(location))+cnt2,7]=file_area_zip["AREALANDPT"].values[0]/1000000
    if year==2011:
        inflation1=inflation[inflation["YEARS"]==year-1]
        inf_rate = inflation1["VALUE"].values[0]
        fileid = pd.read_csv(r'\\nas01.itap.purdue.edu\puhome\My Documents\Research\Paper1\Preprocessing_files\zip\2010_ZiptoLogrecno.csv', header=0)
        file1 = pd.read_csv(r'\\nas01.itap.purdue.edu\puhome\My Documents\Research\Paper1\Preprocessing_files\zip\2011_education.csv', header=0)
        file2 = pd.read_csv(r'\\nas01.itap.purdue.edu\puhome\My Documents\Research\Paper1\Preprocessing_files\zip\2011_last3.csv', header=0)
        file3 = pd.read_csv(r'\\nas01.itap.purdue.edu\puhome\My Documents\Research\Paper1\Preprocessing_files\zip\2011_valuehousehold.csv', header=0)
        file4 = pd.read_csv(r'\\nas01.itap.purdue.edu\puhome\My Documents\Research\Paper1\Preprocessing_files\zip\2010_44.csv', header=0)
        var22=var3[var3["YEAR"]==year-1]
        valuetorent=[]
        for cnt2 in range(0, len(location)):
            data1[(cnt*len(location))+cnt2,0]=year-1
            zipcode = location[cnt2]
            A = zipcodes.matching(str(zipcode))
            county = af.get_county_fips(A[0]["county"], A[0]["state"])
            county=int(county)
            data1[(cnt*len(location))+cnt2,1]=county
            data1[(cnt*len(location))+cnt2,2]=zipcode
            file_ed=file1[file1["ZIP"]==zipcode]
            var222=var22[var22["COUNTY"]==county]
            
            if zipcode==67260:
                continue
            
            if file_ed.shape[0]==1:
                pop_18=float(file_ed["POP1"].values[0])
                pop_24=float(file_ed["POP2"].values[0])
                pop_col=float(file_ed["POP14"].values[0])*pop_18
                for cnt3 in range(24,27):
                    pop_col=pop_col+(float(file_ed["POP"+str(cnt3)].values[0])*pop_24)
                pop_hs=(float(file_ed["POP12"].values[0])*pop_18)+(float(file_ed["POP22"].values[0])*pop_24)
                if (pop_18+pop_24)!=0:
                    data1[(cnt*len(location))+cnt2,3]=pop_col/(pop_18+pop_24)
                    data1[(cnt*len(location))+cnt2,4]=pop_hs/(pop_18+pop_24)
                file_last3=file2[file2["ZIP"]==zipcode]
                if int(file_last3["LF"].values[0])==-999:
                    data1[(cnt*len(location))+cnt2,5]=var222["UNEMPLOYMENT_RATE"].values[0]
                else:
                    data1[(cnt*len(location))+cnt2,5]=(1-(float(file_last3["EMP"].values[0])/(float(file_last3["LF"].values[0]))))*100
                if int(file_last3["INC"].values[0])==-999:
                    data1[(cnt*len(location))+cnt2,8]=float(var222["MED_INC"].values[0])
                else:
                    data1[(cnt*len(location))+cnt2,8]=float(file_last3["INC"].values[0])*inf_rate
                file_val=file3[file3["ZIP"]==zipcode]
                if int(file_val["VALUE"].values[0])==-999:
                    data1[(cnt*len(location))+cnt2,9]=((var222["MED_RENT"].values[0])*np.mean(valuetorent))
                else:
                    data1[(cnt*len(location))+cnt2,9]=float(file_val["VALUE"].values[0])*inf_rate
                    valuetorent.append(float(file_val["VALUE"].values[0])/(var222["MED_RENT"].values[0]))
                file_cnty0=fileid[fileid["ZIPCODE"]==(zipcode)]
                if file_cnty0.shape[0]==1:
                    logrecno = file_cnty0["LOGRECNO"].values[0]
                    file_cnty=file4[file4["LOGRECNO"]==logrecno]
                    if int(file_cnty["OCC"].values[0])!=0:
                        data1[(cnt*len(location))+cnt2,10]=(file_cnty["OWN1"].values[0]+file_cnty["OWN2"].values[0])*100/file_cnty["OCC"].values[0]
                else:
                    data1[(cnt*len(location))+cnt2,10]=var222["HOME_OWNERSHIP"].values[0]
            else:                        
                data1[(cnt*len(location))+cnt2,3]=var222["COLLEGE_COMP"].values[0]
                data1[(cnt*len(location))+cnt2,4]=var222["HIGHSCHOOL_COMP"].values[0]
                data1[(cnt*len(location))+cnt2,5]=var222["UNEMPLOYMENT_RATE"].values[0]
                data1[(cnt*len(location))+cnt2,8]=var222["MED_INC"].values[0]
                data1[(cnt*len(location))+cnt2,9]=((var222["MED_RENT"].values[0])*np.mean(valuetorent))
                file_cnty0=fileid[fileid["ZIPCODE"]==(zipcode)]
                if file_cnty0.shape[0]==1:
                    logrecno = file_cnty0["LOGRECNO"].values[0]
                    file_cnty=file4[file4["LOGRECNO"]==logrecno]
                    data1[(cnt*len(location))+cnt2,10]=(file_cnty["OWN1"].values[0]+file_cnty["OWN2"].values[0])*100/file_cnty["OCC"].values[0]
                else:
                    data1[(cnt*len(location))+cnt2,10]=var222["HOME_OWNERSHIP"].values[0]
            file_air_county=file_air[(file_air["ZIPCODE"]==str(location[cnt2])) & (file_air["YEAR"]<year)]
            data1[(cnt*len(location))+cnt2,6]=int(file_air_county.shape[0])
            file_area_zip=file_area[file_area["ZCTA5"]==zipcode]
            data1[(cnt*len(location))+cnt2,7]=file_area_zip["AREALANDPT"].values[0]/1000000
var2 = pd.DataFrame(data=data1,columns=["YEAR","COUNTY","ZIP","COLLEGE_COMP","HIGHSCHOOL_COMP","UNEMPLOYMENT_RATE","NAIRPORTS","LAND_AREA","MED_INC","MED_VALUE","HOME_OWNERSHIP"])
var2.to_csv(r'\\nas01.itap.purdue.edu\puhome\My Documents\Research\Paper1\Organised Derived CSVs\400_z0var3.csv')

In [5]:
####################        OTHER VARIABLES CONVERSION TO YEARLY (LIN INTERPOLATION)
file=pd.read_csv(r'\\nas01.itap.purdue.edu\puhome\My Documents\Research\Paper1\Organised Derived CSVs\400_z0var3.csv')
var2 = pd.read_csv(r'\\nas01.itap.purdue.edu\puhome\My Documents\Research\Paper1\Organised Derived CSVs\400_var3.csv', header=0)
############  R
data1=np.zeros((file.shape[0],8))
for cnt in range(0,file.shape[0]):
    year=file["YEAR"].values[cnt]
    var20=var2[var2["YEAR"]==year]
    county=file["COUNTY"].values[cnt]
    var21=var20[var20["COUNTY"]==county]
    if var21["COLLEGE_COMP"].values[0]!=0:
        data1[cnt,0]=file["COLLEGE_COMP"].values[cnt]/var21["COLLEGE_COMP"].values[0]
    if var21["HIGHSCHOOL_COMP"].values[0]!=0:
        data1[cnt,1]=file["HIGHSCHOOL_COMP"].values[cnt]/var21["HIGHSCHOOL_COMP"].values[0]
    if var21["UNEMPLOYMENT_RATE"].values[0]!=0:
        data1[cnt,2]=file["UNEMPLOYMENT_RATE"].values[cnt]/var21["UNEMPLOYMENT_RATE"].values[0]
    if var21["NAIRPORTS"].values[0]!=0:
        data1[cnt,3]=file["NAIRPORTS"].values[cnt]/var21["NAIRPORTS"].values[0]
    if var21["LAND_AREA"].values[0]!=0:
        data1[cnt,4]=file["LAND_AREA"].values[cnt]/var21["LAND_AREA"].values[0]
    if var21["MED_INC"].values[0]!=0:
        data1[cnt,5]=file["MED_INC"].values[cnt]/var21["MED_INC"].values[0]
    if var21["MED_RENT"].values[0]!=0:
        data1[cnt,6]=file["MED_VALUE"].values[cnt]/var21["MED_RENT"].values[0]
    if var21["HOME_OWNERSHIP"].values[0]!=0:
        data1[cnt,7]=file["HOME_OWNERSHIP"].values[cnt]/var21["HOME_OWNERSHIP"].values[0]
######   RC
data2=np.zeros((len(location)*3,8))
for cnt in range(0,2):
    for cnt2 in range(0,len(location)):
        data2[(cnt*len(location))+cnt2,0]=(data1[((cnt+1)*len(location))+cnt2,0]-data1[(cnt*len(location))+cnt2,0])*0.11
        data2[(cnt*len(location))+cnt2,1]=(data1[((cnt+1)*len(location))+cnt2,1]-data1[(cnt*len(location))+cnt2,1])*0.11
        data2[(cnt*len(location))+cnt2,2]=(data1[((cnt+1)*len(location))+cnt2,2]-data1[(cnt*len(location))+cnt2,2])*0.11
        data2[(cnt*len(location))+cnt2,3]=(data1[((cnt+1)*len(location))+cnt2,3]-data1[(cnt*len(location))+cnt2,3])*0.11
        data2[(cnt*len(location))+cnt2,4]=(data1[((cnt+1)*len(location))+cnt2,4]-data1[(cnt*len(location))+cnt2,4])*0.11
        data2[(cnt*len(location))+cnt2,5]=(data1[((cnt+1)*len(location))+cnt2,5]-data1[(cnt*len(location))+cnt2,5])*0.11
        data2[(cnt*len(location))+cnt2,6]=(data1[((cnt+1)*len(location))+cnt2,6]-data1[(cnt*len(location))+cnt2,6])*0.11
        data2[(cnt*len(location))+cnt2,7]=(data1[((cnt+1)*len(location))+cnt2,7]-data1[(cnt*len(location))+cnt2,7])*0.1
#######   LINEAR INTERPOLATION
years=[1989,1999,2010]
years_ho=[1989,1999,2009]
cnt0=-1
cnt0_ho=-1
cnt00=-1
initial=1990
data3=np.zeros(((2012-initial+1)*len(location),11))
file_air=pd.read_csv(r"\\nas01.itap.purdue.edu\puhome\My Documents\Research\Paper1\Preprocessing_files\zip\airport.csv",parse_dates=['ActivationDate'])
file_air["YEAR"]=file_air['ActivationDate'].dt.year
for cnt in range(1990,2013):
    year=cnt-1
    index=0
    index_ho=0
    if year in years:
        index=1
        cnt0=cnt0+1
        if year!=years[len(years)-1]:
            cnt00=cnt00+1
        file0=file[file["YEAR"]==year]
        for cnt3 in range(0,10):
            data3[((cnt-initial)*len(location)):((cnt+1-initial)*len(location)),10-cnt3-1]=file0.values[:,file0.shape[1]-1-cnt3-1]
    if year in years_ho:
        index_ho=1
        cnt0_ho=cnt0_ho+1
        file0_ho=file[file["YEAR"]==year]
        if year==years_ho[len(years_ho)-1]:
            file0_ho=file[file["YEAR"]==(year+1)]
        data3[((cnt-initial)*len(location)):((cnt+1-initial)*len(location)),10]=file0_ho.values[:,file0_ho.shape[1]-1]
    if index==1 and index_ho==1:
        continue
    if index==0:
        var20=var2[var2["YEAR"]==year]
        data3[((cnt-initial)*len(location)):((cnt+1-initial)*len(location)),0]=year
        data3[((cnt-initial)*len(location)):((cnt+1-initial)*len(location)),7]=file0["LAND_AREA"].values[:]
    for cnt2 in range(0,len(location)):
        zipcode = location[cnt2]
        A = zipcodes.matching(str(zipcode))
        county = af.get_county_fips(A[0]["county"], A[0]["state"])
        county=int(county)
        var21=var20[var20["COUNTY"]==county]
        if index==0:
            data3[((cnt-initial)*len(location))+cnt2,1]=county
            data3[((cnt-initial)*len(location))+cnt2,2]=zipcode
            data3[((cnt-initial)*len(location))+cnt2,3]=var21["COLLEGE_COMP"].values[0]*(((year-years[cnt0])*data2[(cnt00*len(location))+cnt2,0])+data1[(cnt0*len(location))+cnt2,0])
            if data3[((cnt-initial)*len(location))+cnt2,3]<0:
                data3[((cnt-initial)*len(location))+cnt2,3]=0
            data3[((cnt-initial)*len(location))+cnt2,4]=var21["HIGHSCHOOL_COMP"].values[0]*(((year-years[cnt0])*data2[(cnt00*len(location))+cnt2,1])+data1[(cnt0*len(location))+cnt2,1])
            if data3[((cnt-initial)*len(location))+cnt2,4]<0:
                data3[((cnt-initial)*len(location))+cnt2,4]=0
            data3[((cnt-initial)*len(location))+cnt2,5]=var21["UNEMPLOYMENT_RATE"].values[0]*(((year-years[cnt0])*data2[(cnt00*len(location))+cnt2,2])+data1[(cnt0*len(location))+cnt2,2])
            if data3[((cnt-initial)*len(location))+cnt2,5]<0:
                data3[((cnt-initial)*len(location))+cnt2,5]=0
            data3[((cnt-initial)*len(location))+cnt2,8]=var21["MED_INC"].values[0]*(((year-years[cnt0])*data2[(cnt00*len(location))+cnt2,5])+data1[(cnt0*len(location))+cnt2,5])
            data3[((cnt-initial)*len(location))+cnt2,9]=var21["MED_RENT"].values[0]*(((year-years[cnt0])*data2[(cnt00*len(location))+cnt2,6])+data1[(cnt0*len(location))+cnt2,6])
            file_air_county=file_air[(file_air["ZIPCODE"]==str(zipcode)) & (file_air["YEAR"]<year)]
            data3[((cnt-initial)*len(location))+cnt2,6]=int(file_air_county.shape[0])
        if index_ho==0:
            data3[((cnt-initial)*len(location))+cnt2,10]=var21["HOME_OWNERSHIP"].values[0]*(((year-years[cnt0_ho])*data2[(cnt00*len(location))+cnt2,7])+data1[(cnt0_ho*len(location))+cnt2,7])        
            if data3[((cnt-initial)*len(location))+cnt2,10]<0:
                data3[((cnt-initial)*len(location))+cnt2,10]=0
var2 = pd.DataFrame(data=data3,columns=["YEAR","COUNTY","ZIP","COLLEGE_COMP","HIGHSCHOOL_COMP","UNEMPLOYMENT_RATE","NAIRPORTS","LAND_AREA","MED_INC","MED_VALUE","HOME_OWNERSHIP"])
var2.to_csv(r'\\nas01.itap.purdue.edu\puhome\My Documents\Research\Paper1\Organised Derived CSVs\400_zvar3.csv')

In [7]:
####################    WAGE
var4 = pd.read_csv(r'\\nas01.itap.purdue.edu\puhome\My Documents\Research\Paper1\Organised Derived CSVs\400_var4.csv')
ngroups=5
initial=1990
data1 = np.zeros((len(location)*(2012-initial+1)*ngroups,5),dtype=float)
for cnt in range(initial,2013):
    var44 = var4[var4["YEAR"]==cnt-1]
    for cnt2 in range(0,len(location)):
        zipcode = location[cnt2]
        A = zipcodes.matching(str(zipcode))
        county = af.get_county_fips(A[0]["county"], A[0]["state"])
        file1 = var44[(var44["COUNTY"]==int(county))]
        for cnt3 in range(0,ngroups):
            data1[((cnt-initial)*len(location)*ngroups)+(cnt2*ngroups)+cnt3,0]=cnt-1
            data1[((cnt-initial)*len(location)*ngroups)+(cnt2*ngroups)+cnt3,1]=county
            data1[((cnt-initial)*len(location)*ngroups)+(cnt2*ngroups)+cnt3,2]=zipcode
            data1[((cnt-initial)*len(location)*ngroups)+(cnt2*ngroups)+cnt3,3]=cnt3+1
            file2 = file1[(file1["NGROUP"]==cnt3+1)]
            if file2.shape[0]!=0:
                wage = file2.iloc[0]["AVG_WAGE"]
            else:
                wage = data1[((cnt-initial-1)*len(location)*ngroups)+(cnt2*ngroups)+cnt3,4]
            data1[((cnt-initial)*len(location)*ngroups)+(cnt2*ngroups)+cnt3,4]=wage
var2 = pd.DataFrame(data=data1,columns=["YEAR","COUNTY","ZIP","NGROUP","AVG_WAGE"])
var2.to_csv(r'\\nas01.itap.purdue.edu\puhome\My Documents\Research\Paper1\Organised Derived CSVs\400_zvar4.csv')

In [3]:
####################    WAGE 20 GROUPS
var4 = pd.read_csv(r'\\nas01.itap.purdue.edu\puhome\My Documents\Research\Paper1\Organised Derived CSVs\20_400_var4.csv')
ngroups=20
initial=1990
data1 = np.zeros((len(location)*(2012-initial+1)*ngroups,5),dtype=float)
for cnt in range(initial,2013):
    var44 = var4[var4["YEAR"]==cnt-1]
    for cnt2 in range(0,len(location)):
        zipcode = location[cnt2]
        A = zipcodes.matching(str(zipcode))
        county = af.get_county_fips(A[0]["county"], A[0]["state"])
        file1 = var44[(var44["COUNTY"]==int(county))]
        for cnt3 in range(0,ngroups):
            data1[((cnt-initial)*len(location)*ngroups)+(cnt2*ngroups)+cnt3,0]=cnt-1
            data1[((cnt-initial)*len(location)*ngroups)+(cnt2*ngroups)+cnt3,1]=county
            data1[((cnt-initial)*len(location)*ngroups)+(cnt2*ngroups)+cnt3,2]=zipcode
            data1[((cnt-initial)*len(location)*ngroups)+(cnt2*ngroups)+cnt3,3]=cnt3+1
            file2 = file1[(file1["NGROUP"]==cnt3+1)]
            if file2.shape[0]!=0:
                wage = file2.iloc[0]["AVG_WAGE"]
            else:
                wage = data1[((cnt-initial-1)*len(location)*ngroups)+(cnt2*ngroups)+cnt3,4]
            data1[((cnt-initial)*len(location)*ngroups)+(cnt2*ngroups)+cnt3,4]=wage
var2 = pd.DataFrame(data=data1,columns=["YEAR","COUNTY","ZIP","NGROUP","AVG_WAGE"])
var2.to_csv(r'\\nas01.itap.purdue.edu\puhome\My Documents\Research\Paper1\Organised Derived CSVs\20_400_zvar4.csv')